In [65]:
import numpy as np

def get_mat(file):
    mat = []
    answ = []
    with open(file) as file_handler:
        for line in file_handler:
            row = list(map(int, line.split(',')[:-1]))
            mat.append(row[:-1])
            answ.append([row[-1]])
        mat = np.asarray(mat)
        answ = np.asarray(answ).reshape(-1,1)
    return np.matrix(mat), np.matrix(answ)

def get_LU(mat):
    lu_mat = np.matrix(np.zeros([mat.shape[0],mat.shape[1]]))
    n = mat.shape[0]
    for k in range(n):
        for j in range(k, mat.shape[1]):
            lu_mat[k, j] = mat[k, j] - lu_mat[k, :k] * lu_mat[:k, j]
        for i in range(k+1, n):
            lu_mat[i,k] = (mat[i,k] - lu_mat[i, :k] * lu_mat[:k, k]) / lu_mat[k, k]
    return np.matrix(lu_mat)


def get_L(lu_mat):
    L = lu_mat.copy()
    for i in range(L.shape[0]):
        L[i, i] = 1
        L[i, i+1:] = 0
    return np.matrix(L)

def get_U(lu_mat):
    U = lu_mat.copy()
    for i in range(1, U.shape[0]):
        U[i, :i] = 0
    return np.matrix(U)

def LU_decomposition(mat):
    mat = np.matrix(mat)
    lu = get_LU(mat)
    matL = get_L(lu)
    matU = get_U(lu)
    return L, U

"""
def LU_decomposition(A):
    n = len(A)
    C = A.copy()
    P = np.arange(0,n)
    p=0
    for i in range(n):
        pivotValue = 0;
        pivot = -1;
        for row in range(i,n) :
            if( abs(C[row][i]) > pivotValue ):
                pivotValue = abs(C[row][i]);
                pivot = row;
        if( pivotValue != 0 ):
            p=p+1;
            C[i][:], C[pivot][:] = C[pivot][:], C[i][:]
            P[i], P[pivot] = P[pivot], P[i]
            for j in range(i+1,n): 
                C[j][i] /= C[i][i]
                for k in range(i+1,n): 
                    C[j][k] -= C[j][i] * C[i][k]
    matU = np.zeros((n,n))
    matL = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if(i<j):
                matU[i][j] = C[i][j]
            if(i==j):
                matU[i][j] = C[i][j]
                matL[i][j] = 1;
            if(i>j):
                matL[i][j] = C[i][j]
    return matL, matU, P, p

def solve(L, U, b):
    n = len(L)
    z = np.zeros(n)
    z[0] = b[0]
    for i in range(1, n):
        tmp = 0;
        for j in range(i):
            tmp += L[i][j]*z[j]
        z[i] = b[i] - tmp
    x = np.zeros(n)
    x[n-1] = z[n-1] / U[n-1][n-1]
    for i in range(n-2, -1,-1):
        tmp = 0;
        for j in range(i+1, n):
            tmp += U[i][j] * x[j]
        x[i] = (z[i] - tmp)/U[i][i]
    return x;

def invers(L, U, P):
    n = len(L)
    res = np.zeros((n,n))
    res1 = np.zeros((n,n))
    E = np.eye(n)
    for i in range(n):
        y = solve(L, E, E[i,:])
        x = solve(E, U, y)
        for j in range(n):
            res[j,i] = x[j]
    for i in range(n):
        res1[:,P[i]] = res[:,i]
    return res1
"""

def solve(lu_mat,b):

    y = np.matrix(np.zeros([lu_mat.shape[0], 1]))
    for i in range(y.shape[0]):
        y[i, 0] = b[i, 0] - lu_mat[i, :i] * y[:i]
        x = np.matrix(np.zeros([lu_mat.shape[0], 1]))
    for i in range(1, x.shape[0] + 1):
        x[-i, 0] = (y[-i] - lu_mat[-i, -i:] * x[-i:, 0] )/ lu_mat[-i, -i]
    return x


def invers(LU):
    E = np.matrix(np.eye(len(LU)))
    new_LU = LU.copy()
    for i in range(len(E)):
        new_LU[:,i] = solve(LU,E[:, i])
    return(new_LU)


def is_zero(mat,b):
    if np.allclose(b, np.zeros([b.shape[0],1])):
        b = mat[:, mat.shape[1]-1]
        mat = np.delete(mat, mat.shape[1]-1, 1)
    return(mat,b)


def det(U):
    determ = 1
    for i in range(U.shape[0]):
        determ *= U[i,i]

    return determ


A,answ = get_mat("test1.txt")
print("A\n",A)
print("B\n", answ)
L,U = LU_decomposition(A.tolist())
print("L\n",L)
print("U\n",U)
lu = get_LU(A)
print("x\n",solve(lu,answ))
print("Проверка корней AX = B:\n", np.dot(A,solve(lu,answ)))
print("Обратная матрица:\n", invers(lu))
print("Проверка обратной матрицы AA^-1 = E:\n",np.dot(invers(lu),A))
print("Определитель:\n", det(U))

A
 [[-5 -1 -3 -1]
 [-2  0  8 -4]
 [-7 -2  2 -2]
 [ 2 -4 -4  4]]
B
 [[ 18]
 [-12]
 [  6]
 [-12]]
L
 [[ 1.          0.          0.          0.        ]
 [-0.28571429  1.          0.          0.        ]
 [ 0.28571429 -0.125       1.          0.        ]
 [ 0.71428571 -0.09375    -0.67857143  1.        ]]
U
 [[-7.         -2.          2.         -2.        ]
 [ 0.         -4.57142857 -3.42857143  3.42857143]
 [ 0.          0.          7.         -3.        ]
 [ 0.          0.          0.         -1.28571429]]
x
 [[-2.]
 [ 3.]
 [-3.]
 [-2.]]
Проверка корней AX = B:
 [[ 18.]
 [-12.]
 [  6.]
 [-12.]]
Обратная матрица:
 [[ 0.22222222  0.22222222 -0.33333333  0.11111111]
 [-0.33333333 -0.33333333  0.25       -0.29166667]
 [-0.33333333 -0.08333333  0.25       -0.04166667]
 [-0.77777778 -0.52777778  0.66666667 -0.13888889]]
Проверка обратной матрицы AA^-1 = E:
 [[ 1.00000000e+00  3.33066907e-16  2.22044605e-15 -1.11022302e-16]
 [-3.66373598e-15  1.00000000e+00 -7.99360578e-15  2.77555756e-16]
 [

In [66]:
A,answ = get_mat("test1.txt")
print("A\n",A)
print("B\n", answ)
L,U = LU_decomposition(A.tolist())
print("L\n",L)
print("U\n",U)
lu = get_LU(A)
print("x\n",solve(lu,answ))
print("Проверка корней AX = B:\n", np.dot(A,solve(lu,answ)))
print("Обратная матрица:\n", invers(lu))
print("Проверка обратной матрицы AA^-1 = E:\n",np.dot(invers(lu),A))
print("Определитель:\n", det(U))

A
 [[-5 -1 -3 -1]
 [-2  0  8 -4]
 [-7 -2  2 -2]
 [ 2 -4 -4  4]]
B
 [[ 18]
 [-12]
 [  6]
 [-12]]
L
 [[ 1.          0.          0.          0.        ]
 [-0.28571429  1.          0.          0.        ]
 [ 0.28571429 -0.125       1.          0.        ]
 [ 0.71428571 -0.09375    -0.67857143  1.        ]]
U
 [[-7.         -2.          2.         -2.        ]
 [ 0.         -4.57142857 -3.42857143  3.42857143]
 [ 0.          0.          7.         -3.        ]
 [ 0.          0.          0.         -1.28571429]]
x
 [[-2.]
 [ 3.]
 [-3.]
 [-2.]]
Проверка корней AX = B:
 [[ 18.]
 [-12.]
 [  6.]
 [-12.]]
Обратная матрица:
 [[ 0.22222222  0.22222222 -0.33333333  0.11111111]
 [-0.33333333 -0.33333333  0.25       -0.29166667]
 [-0.33333333 -0.08333333  0.25       -0.04166667]
 [-0.77777778 -0.52777778  0.66666667 -0.13888889]]
Проверка обратной матрицы AA^-1 = E:
 [[ 1.00000000e+00  3.33066907e-16  2.22044605e-15 -1.11022302e-16]
 [-3.66373598e-15  1.00000000e+00 -7.99360578e-15  2.77555756e-16]
 [